In [ ]:
from flask import Flask, jsonify, request
from flask_cors import CORS
import threading
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
import os
from flask_restx import Api, Resource
import logging

# Инициализация приложения Flask и настройка CORS и API
app = Flask(__name__)
CORS(app)
api = Api(app, version='1.0', title='Jupyter Notebook Runner API', description='An API to run Jupyter notebooks')

# Настройка логирования
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger()

# Словарь для хранения состояния выполнения для каждого этапа
progress = {
    'run_scrape': 0,
    'run_summarization': 0,
    'run_grade': 0,
    'run_digest_generation': 0
}

# Функция для выполнения Jupyter notebook
def execute_notebook(notebook_paths, progress_key):
    progress[progress_key] = 0
    try:
        for notebook_path in notebook_paths:
            logger.info(f"Starting execution of notebook: {notebook_path}")
            with open(notebook_path) as f:
                nb = nbformat.read(f, as_version=4)
            ep = ExecutePreprocessor(timeout=600, kernel_name='python3')
            ep.preprocess(nb, {'metadata': {'path': './'}})
            logger.info(f"Successfully executed notebook: {notebook_path}")
        progress[progress_key] = 100
    except Exception as e:
        progress[progress_key] = -1
        logger.error(f"Failed to execute notebook {notebook_path}: {e}", exc_info=True)


@api.route('/run_scrape')
class RunScrape(Resource):
    @api.doc(params={'token': 'Authorization token'})
    @api.response(200, 'Success')
    @api.response(400, 'Notebook not found')
    @api.response(403, 'Invalid token')
    def get(self):
        token = request.args.get('token')
        if token != 'your_secret_token':
            return {'error': 'Invalid token'}, 403

        notebook_paths = ['rss.ipynb', 'scrape.ipynb', 'news_extraction.ipynb']  # Укажите пути к вашим ноутбукам
        for path in notebook_paths:
            if not os.path.exists(path):
                return {'error': f'Notebook {path} not found'}, 400

        # Запускаем выполнение ноутбуков в отдельном потоке, чтобы не блокировать основной поток
        thread = threading.Thread(target=execute_notebook, args=(notebook_paths, 'run_scrape'))
        thread.start()
        
        return {
            'status': 'Scrape notebook execution started',
            'notebooks': notebook_paths,
            'message': 'Please check results: https://docs.google.com/spreadsheets/d/1BA1nioQqc048FFvKXcpP5VqL_73kXNCcSx0m-jhI2MQ/edit?gid=2038294867#gid=2038294867'
        }, 200

@api.route('/run_summarization')
class RunSummarization(Resource):
    @api.doc(params={'token': 'Authorization token'})
    @api.response(200, 'Success')
    @api.response(400, 'Notebook not found')
    @api.response(403, 'Invalid token')
    def get(self):
        token = request.args.get('token')
        if token != 'your_secret_token':
            return {'error': 'Invalid token'}, 403

        notebook_path = 'news_range_n_summary.ipynb'  # Укажите путь к вашему ноутбуку news_range_n_summary.ipynb

        if not os.path.exists(notebook_path):
            return {'error': 'Notebook not found'}, 400

        # Запускаем выполнение ноутбука в отдельном потоке, чтобы не блокировать основной поток
        thread = threading.Thread(target=execute_notebook, args=([notebook_path], 'run_summarization'))
        thread.start()
        
        return {
            'status': 'Summarization notebook execution started',
            'notebook': notebook_path,
            'message': 'Please check results: https://docs.google.com/spreadsheets/d/1BA1nioQqc048FFvKXcpP5VqL_73kXNCcSx0m-jhI2MQ/edit?gid=2038294867#gid=2038294867'
        }, 200

@api.route('/run_grade')
class RunGrade(Resource):
    @api.doc(params={'token': 'Authorization token'})
    @api.response(200, 'Success')
    @api.response(400, 'Notebook not found')
    @api.response(403, 'Invalid token')
    def get(self):
        token = request.args.get('token')
        if token != 'your_secret_token':
            return {'error': 'Invalid token'}, 403

        notebook_path = 'news_range_n_summary.ipynb'  # Укажите путь к вашему ноутбуку news_range_n_summary.ipynb

        if not os.path.exists(notebook_path):
            return {'error': 'Notebook not found'}, 400

        # Запускаем выполнение ноутбука в отдельном потоке, чтобы не блокировать основной поток
        thread = threading.Thread(target=execute_notebook, args=([notebook_path], 'run_grade'))
        thread.start()
        
        return {
            'status': 'Grade notebook execution started',
            'notebook': notebook_path,
            'message': 'Please check results: https://docs.google.com/spreadsheets/d/1BA1nioQqc048FFvKXcpP5VqL_73kXNCcSx0m-jhI2MQ/edit?gid=2038294867#gid=2038294867'
        }, 200

@api.route('/run_digest_generation')
class RunDigestGeneration(Resource):
    @api.doc(params={'token': 'Authorization token'})
    @api.response(200, 'Success')
    @api.response(400, 'Notebook not found')
    @api.response(403, 'Invalid token')
    def get(self):
        token = request.args.get('token')
        if token != 'your_secret_token':
            return {'error': 'Invalid token'}, 403

        notebook_path = 'digest.ipynb'  # Укажите путь к вашему ноутбуку digest.ipynb

        if not os.path.exists(notebook_path):
            return {'error': 'Notebook not found'}, 400

        # Запускаем выполнение ноутбука в отдельном потоке, чтобы не блокировать основной поток
        thread = threading.Thread(target=execute_notebook, args=([notebook_path], 'run_digest_generation'))
        thread.start()
        
        return {
            'status': 'Digest generation notebook execution started',
            'notebook': notebook_path,
            'message': 'Please check results: https://docs.google.com/spreadsheets/d/1BA1nioQqc048FFvKXcpP5VqL_73kXNCcSx0m-jhI2MQ/edit?gid=2038294867#gid=2038294867'
        }, 200

@api.route('/progress/<string:step>')
class GetProgress(Resource):
    @api.doc(params={'token': 'Authorization token'})
    @api.response(200, 'Success')
    @api.response(400, 'Invalid step')
    @api.response(403, 'Invalid token')
    def get(self, step):
        token = request.args.get('token')
        if token != 'your_secret_token':
            return {'error': 'Invalid token'}, 403

        if step not in progress:
            return {'error': 'Invalid step'}, 400

        return {
            'step': step,
            'progress': progress[step]
        }, 200

if __name__ == '__main__':
    app.run(host='45.133.178.134', port=35474)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://45.133.178.134:35474
INFO:werkzeug:Press CTRL+C to quit
INFO:root:Starting execution of notebook: news_range_n_summary.ipynb
INFO:werkzeug:185.155.16.157 - - [14/Jun/2024 14:00:28] "GET /run_summarization?token=your_secret_token HTTP/1.1" 200 -
DEBUG:asyncio:Using selector: EpollSelector
DEBUG:asyncio:Using selector: EpollSelector
INFO:root:Successfully executed notebook: news_range_n_summary.ipynb
INFO:root:Starting execution of notebook: digest.ipynb
INFO:werkzeug:185.155.16.157 - - [14/Jun/2024 14:01:40] "GET /run_digest_generation?token=your_secret_token HTTP/1.1" 200 -
DEBUG:asyncio:Using selector: EpollSelector
DEBUG:asyncio:Using selector: EpollSelector
INFO:root:Successfully executed notebook: digest.ipynb


In [ ]:
execute_notebook('rss.ipynb')

In [ ]:
%run rss.ipynb

In [ ]:
!pip install flask-restx


